<a href="https://colab.research.google.com/github/MicroPyscho/MicroPyscho/blob/main/ANN_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [244]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from google.colab import drive
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from torch import nn

Load dataset and replace all +ve and -ve infinite values with NaN

In [ ]:
star_df = pd.read_csv("/content/drive/MyDrive/Star_df dataset").replace([np.inf, -np.inf], np.nan)

if "Unnamed: 0" in star_df.columns:
  star_df.drop(columns =["Unnamed: 0"], inplace=True)

In [ ]:
unique_values = star_df['ficincome'].unique()
print(unique_values)

In [247]:
var_sum = ['_CGSRDP049', '_CGVSDP047', '_CGVSDP048', '_CGVSDP049',
                    '_CGVSDP050', '_CGVSDP053', '_CGVSDP054', '_CGVSDP055', '_CGVSDP056', '_CGVSDP057', '_CGVSDP058']
star_df['SDG5_9&12_17'] = star_df[var_sum].sum(axis=1)
star_df.drop(columns=var_sum, inplace=True)

In [248]:
var_sum2 = ['_ENERDP062', '_ENERDP063', '_ENERDP068', "Target", "CCIndexd1", "SUS_report", "ENV_committee"]
star_df['waste_redctn_efforts'] = (star_df[var_sum2].sum(axis=1) / 7).clip(0, 1)
star_df.drop(columns=var_sum2, inplace=True)


In [249]:
star_df['_ENRRO04V'] = (star_df['_ENRRO04V'] - star_df['_ENRRO04V'].mean()) / star_df['_ENRRO04V'].std()
var_sum3 = ["_ENPIDP066", "_ENPIDP069", "_ENRRO04V", "EcoInn_score"]
star_df['clean_tech_efrts'] = (star_df[var_sum3].sum(axis=1) / 4).clip(0, 1)
star_df.drop(columns=var_sum3, inplace=True)

In [250]:
var_sum4 = ["_SOCODP011", "_SOCOO10V", "CrisisMgt"]
star_df['anomaly_reportng_efrts'] = (star_df[var_sum4].sum(axis=1) / 3).clip(0, 1)
star_df.drop(columns=var_sum4, inplace=True)

In [251]:
star_df['_SODODP017'] = (star_df['_SODODP017'] - star_df['_SODODP017'].mean()) / star_df['_SODODP017'].std()
star_df['_SODODP019'] = (star_df['_SODODP019'] - star_df['_SODODP019'].mean()) / star_df['_SODODP019'].std()
var_sum5 = ["_SODODP019", "_SODODP017"]
star_df['women_rpresntn'] = (star_df[var_sum5].sum(axis=1) / 2).clip(0, 1)
star_df.drop(columns=var_sum5, inplace=True)

In [252]:
var_sum6= ['climatevul_capacity', 'climatevul_exposure', 'climatevul_sensitivity',
                    'climate_vul', 'climate_economic', 'climate_governance',
                    'climate_readiness', 'climate_social']
star_df['climate_preparedness'] = star_df[var_sum6].sum(axis=1)
star_df.drop(columns=var_sum6, inplace=True)

In [253]:
var_avg = ['CO2eGDP', 'CO2eGDP_PPP2017', 'CO2eGDP_PPP']
star_df['avgCO2_GDP'] = star_df[var_avg].mean(axis=1)
star_df.drop(columns=var_avg, inplace=True)

In [254]:
star_df['bal_of_tradeGDP%'] = star_df['ExportsGDP'] - star_df['ImportsGDP']
var_std = ['GDP_K2015', 'GDPg', 'GDP_percap2015', 'GDPg_percap', "MktCapGDP", "GovtDebtGDP", "bal_of_tradeGDP%", "TradeGDP"]
for var in var_std:
    star_df[f'{var}_standardized'] = (star_df[var] - star_df[var].mean()) / star_df[var].std()
star_df['gdp_prfmnce'] = star_df[[f'{var}_standardized' for var in var_std]].sum(axis=1)
star_df['gdp_prfmnce_rank'] = star_df['gdp_prfmnce'].rank(pct=True) * 100
star_df.drop(columns=var_std, inplace=True)

In [255]:
income_class_map = {'High income': 4, 'Upper middle income': 3, 'Lower middle income': 2, 'Low income': 1}
star_df['incme_class'] = star_df['ficincome'].map(income_class_map)
var_std2 = ['var', 'pvr', 'ger', 'rqr', 'rlr', 'ccr', 'hdi_score', 'incme_class', "gain"]
for var in var_std2:
  star_df[f'{var}_standardized'] = (star_df[var] - star_df[var].mean()) / star_df[var].std()
star_df['eco_soc_gov_fctrs'] = star_df[[f'{var}_standardized' for var in var_std2]].sum(axis=1)
star_df['eco_soc_gov_fctrs'] = (star_df['eco_soc_gov_fctrs'] - star_df['eco_soc_gov_fctrs'].min()) / (star_df['eco_soc_gov_fctrs'].max() - star_df['eco_soc_gov_fctrs'].min()) * 100
star_df.drop(columns=var_std2, inplace=True)

In [256]:
star_df['dvsrty&inclsn_score'] = (star_df['_TRDIRDS'] + star_df['_TRDIRIS']) / 2

star_df['total_emissns'] = star_df['_ENERDP023'] + star_df['_ENERDP123'] + star_df["_ENERO03V"]
star_df['Net_FDI_GDP'] = star_df['FDI_INFGDP'] - star_df['FDI_OUTGDP']

star_df['avg_EPS'] = (star_df['_EPS'] + star_df['_EPS2MN']) / 2
star_df['DCred2PrvSctr_GDP'] = (star_df['DCredPvtGDP'] - star_df['DCredPvtGDP'].mean()) / star_df['DCredPvtGDP'].std()
star_df['agg_net_cash_flow'] = star_df['_WC04860'] + star_df['_WC04870'] + star_df['_WC04890']
star_df['operating_profit'] = star_df['_WC01250'] - star_df['_WC01249']
star_df["abs_sales_growth"] = star_df['sgrowth'] - star_df['sicsgrowth']
star_df.rename(columns={'_WC03151': 'working_capital'}, inplace=True)
star_df.rename(columns={"_WC01001": "net_revenue"}, inplace=True)
star_df.rename(columns={"_WC01201": "R&D"}, inplace=True)
star_df.rename(columns={"_WC03501": "common_equity"}, inplace=True)
star_df.rename(columns={"_WC03998": "total_capital"}, inplace=True)
star_df.rename(columns={"_WC08106": "current_ratio"}, inplace=True)
star_df.rename(columns={"_WC08126": "invntry_hold_days"}, inplace=True)
star_df.rename(columns={"_WC08366": "net_prfit_margin"}, inplace=True)
star_df.rename(columns={"roe": "retrn_on_equity"}, inplace=True)
star_df.rename(columns={"_CGBSO19": "board_gndr_div%"}, inplace=True)
star_df.rename(columns={"_CGSCORE": "govnce_pilr_score"}, inplace=True)
star_df.rename(columns={"_CGVSDP046": "govnce_pilr_score"}, inplace=True)
star_df.rename(columns={"_ENERDP024": "dirct_cemissions"}, inplace=True)
star_df.rename(columns={"_ENERDP025": "indirct_cemissions"}, inplace=True)
star_df.rename(columns={"_ENERO27V": "scope1"}, inplace=True)
star_df.rename(columns={"_ENERO28V": "scope2"}, inplace=True)
star_df.rename(columns={"_ENRRDP033": "total_enrgy_use"}, inplace=True)
star_df.rename(columns={"_ENRRDP046": "renwbl_enrgy_use"}, inplace=True)
star_df.rename(columns={"_ENSCORE": "envrmt_pilr_score"}, inplace=True)
star_df.rename(columns={"_TRESGCS": "ESG_score"}, inplace=True)
star_df.rename(columns={"_TRESGCGVSS": "CSR_score"}, inplace=True)
star_df.rename(columns={"_TRESGENERS": "emissn_score"}, inplace=True)
star_df.rename(columns={"INTSpread": "Intrst_rate_spread"}, inplace=True)
star_df.rename(columns={"pop_score": "population"}, inplace=True)


#star_df['working_capital'] = star_df['_WC02201'] - star_df['_WC02999']

In [257]:
list_A = [
    'DSCD', 'ticker', 'exchange', 'WC06011', 'WC07040', '_EPS', '_EPS2MN', '_MVC',
    '_P', '_WC01051', '_WC01084', '_WC01249', '_WC01100', '_WC01250', '_WC01401',
    '_WC02001', '_WC02051', '_WC02101', '_WC02201', '_WC02501', '_WC02999',
    '_WC03101', '_WC03255', '_WC03351', '_WC03480', '_WC03995', '_WC04551',
    '_WC04601', '_WC04860', '_WC04870', '_WC04890', '_WC05201', '_WC05476',
    '_WC05502', '_WC08101', '_WC08321', '_WC08301', '_WC08316', '_WC08366',
    '_WC09504', '_WC18193', '_WC18199', '_WC18198', 'ffi5', 'cashna', 'cash2sales',
    'ppe', 'Size', 'logsales', 'logppe', 'firmage37', 'logage', 'tlta', 'tdta',
    'tdmv', 'clta', 'cata', 'apta', 'arta', 'arsale', 'apcogs', 'nettcsale', 'dep',
    'roa', 'ros', 'ebitta', 'tobinqe', 'mb', 'capexppe', 'dcashta', 'dcashsales',
    'dcashna', 'sgrowth', 'sicsgrowth', 'nwccta', 'divta', 'tri', 'divchg', 'lppe',
    'lsize', 'llogsales', 'lcfw', 'lroa', 'lroe', 'ltobinq', 'lcapexppe', 'larta',
    'lapta', 'llogemp', 'ldep', 'llogage', 'llogage2', 'lcashna', 'lcashsales',
    'lcash2ta', 'lempgrowth', 'lcapexrdta', 'lcash2sales', 'lcash2na', 'wwindex',
    'kzindex', 'saindex', 'mwail', 'zscore', 'zscoree', '_CGSRDP049', '_CGVSDP046',
    '_CGVSDP047', '_CGVSDP048', '_CGVSDP049', '_CGVSDP050', '_CGVSDP053',
    '_CGVSDP054', '_CGVSDP055', '_CGVSDP056', '_CGVSDP057', '_CGVSDP058',
    '_ENERDP033', '_ENERDP062', '_ENERO32V', '_SOCODP037', '_SODODP0151',
    '_SOEQO08V', '_TRDIRDs', '_TRDIRIS', '_TRESGS', '_TRESGENPIS', 'TCE', 'DCE',
    'INDCE', 'logTCE', 'logDCE', 'logINDCE', 'logEMScore', 'TCETA', 'DCETA',
    'ENVInitiatives', 'logENVInvScore', 'EnergyUse', 'RenEnergyUse',
    'EnergyUseSale', 'logEnergyUse', 'logResUse_score', 'logMgtScore',
    'logSHScore', 'logCSRScore', 'logCMScore', 'logHRScore', 'logPRScore',
    'GOVScore', 'SOSScore', 'ESG_Comp', 'MgersFem', 'BODDIV', 'EXEDIV',
    'SnrExecCompTA', 'SnrExecCompSale', 'CrisisMgt', 'EmissionsTrdg', 'SDG13',
    'Climate_gov', 'ENV_Prod', 'ENV_Asset', 'Eco_Prod', 'RespUse_Prod',
    'CleanTech', 'EcoInn_score', 'ENV_train', 'MgtTrg', 'ESGC', 'CO2eGDP',
    'CO2eGDP_PPP2017', 'CO2eGDP_PPP', 'FDI_OUTGDP', 'FDI_INFGDP', 'DCredPvtGDP',
    'GDP_Cur', 'GDP_K2015', 'GDPg', 'GDP_percap2015', 'GDPg_percap', 'MktCapGDP',
    'GovtDebtGDP', 'bal_of_tradeGDP%', 'INFL_conpx', 'INTPayExp', 'INTPayRev',
    'RealINT', 'gdp_score', 'var', 'pvr', 'ger', 'rqr', 'rlr', 'ccr', 'hdi_score',
    'incme_class', 'gain', 'ficincome', 'climatevul_capacity', 'climatevul_exposure',
    'climatevul_sensitivity', 'climate_vul', 'climate_economic', 'climate_governance',
    'climate_readiness', 'climate_social'
]

list_B = [
    '_CGSRDP049', '_CGVSDP047', '_CGVSDP048', '_CGVSDP049',
    '_CGVSDP050', '_CGVSDP053', '_CGVSDP054', '_CGVSDP055', '_CGVSDP056', '_CGVSDP057', '_CGVSDP058', '_ENERDP062', '_ENERDP063', '_ENERDP068', "Target", "CCIndexd1", "SUS_report", "ENV_committee", "_ENPIDP066", "_ENPIDP069", "_ENRRO04V", "EcoInn_score", "_SOCODP011", "_SOCOO10V", "CrisisMgt", "_SODODP019", "_SODODP017", 'climatevul_capacity', 'climatevul_exposure', 'climatevul_sensitivity',
                    'climate_vul', 'climate_economic', 'climate_governance',
                    'climate_readiness', 'climate_social', 'CO2eGDP', 'CO2eGDP_PPP2017', 'CO2eGDP_PPP',  'GDP_K2015', 'GDPg', 'GDP_percap2015', 'GDPg_percap', "MktCapGDP", "GovtDebtGDP", "bal_of_tradeGDP%", "TradeGDP", 'var', 'pvr', 'ger', 'rqr', 'rlr', 'ccr', 'hdi_score', 'incme_class', "gain", "_ENERO03V", '_ENERDP123', '_ENERDP023', '_TRDIRIS', '_TRDIRDS', 'FDI_OUTGDP', 'FDI_INFGDP', '_EPS2MN', '_EPS', 'DCredPvtGDP', '_WC04870', '_WC01249', '_WC01250', '_WC04890', '_WC04860', 'sicsgrowth', 'sgrowth'
]

# Remove duplicates from list_B in list_A
list_A = [variable for variable in list_A if variable not in list_B]

print(list_A)


['DSCD', 'ticker', 'exchange', 'WC06011', 'WC07040', '_MVC', '_P', '_WC01051', '_WC01084', '_WC01100', '_WC01401', '_WC02001', '_WC02051', '_WC02101', '_WC02201', '_WC02501', '_WC02999', '_WC03101', '_WC03255', '_WC03351', '_WC03480', '_WC03995', '_WC04551', '_WC04601', '_WC05201', '_WC05476', '_WC05502', '_WC08101', '_WC08321', '_WC08301', '_WC08316', '_WC08366', '_WC09504', '_WC18193', '_WC18199', '_WC18198', 'ffi5', 'cashna', 'cash2sales', 'ppe', 'Size', 'logsales', 'logppe', 'firmage37', 'logage', 'tlta', 'tdta', 'tdmv', 'clta', 'cata', 'apta', 'arta', 'arsale', 'apcogs', 'nettcsale', 'dep', 'roa', 'ros', 'ebitta', 'tobinqe', 'mb', 'capexppe', 'dcashta', 'dcashsales', 'dcashna', 'nwccta', 'divta', 'tri', 'divchg', 'lppe', 'lsize', 'llogsales', 'lcfw', 'lroa', 'lroe', 'ltobinq', 'lcapexppe', 'larta', 'lapta', 'llogemp', 'ldep', 'llogage', 'llogage2', 'lcashna', 'lcashsales', 'lcash2ta', 'lempgrowth', 'lcapexrdta', 'lcash2sales', 'lcash2na', 'wwindex', 'kzindex', 'saindex', 'mwail', 

In [ ]:
col_dp = ['DSCD', 'ticker', 'exchange', 'WC06011', 'WC07040', '_MVC', '_P', '_WC01051', '_WC01084', '_WC01100']
star_df.drop(columns=col_dp, inplace=True)

In [ ]:
# Drop variables in list_A from star_df
star_df.drop(columns=list_A, inplace=True)

In [260]:
star_df = star_df.loc[:, ~star_df.columns.str.endswith('_standardized')]

In [ ]:
for col_name, count in star_df.count().items():
    print(f"{col_name}: {count}")

In [ ]:
# First, create a DataFrame to store the log-transformed variables
log_star_df = pd.DataFrame()

# Iterate through each column in star_df
for col in star_df.columns:
    # Check if the column contains numerical data
    if np.issubdtype(star_df[col].dtype, np.number):
        # Apply log transformation and add it to log_star_df
        log_star_df[col+'_log'] = np.log(star_df[col])
    else:
        # If the column contains non-numerical data, simply copy it to log_star_df
        log_star_df[col] = star_df[col]

# Now log_star_df contains log-transformed variables (for numerical columns)


In [ ]:
log_star_df.to_csv('/content/drive/My Drive/log_star_df.csv', index=False)

In [ ]:
for col_name, count in log_star_df.count().items():
    print(f"{col_name}: {count}")

Exclude all object dtype variable in preparation for ANN and CNN model

In [ ]:
#star_df = star_df.select_dtypes(exclude=['object'])

In [ ]:
star_df.head()

In [ ]:
star_df.shape

Replace any residual "-" in any variable with the mean of the variable column

In [ ]:
star_df.replace('-', pd.NA, inplace=True)
star_df = star_df.apply(pd.to_numeric, errors='coerce')
column_means = star_df.mean(skipna=True)
star_df.fillna(column_means, inplace=True)

print("Updated DataFrame:")
print(star_df)


Split the variables according to their dtypes and note them

In [ ]:
obj_cols = star_df.select_dtypes(include=['object'])

int_cols = star_df.select_dtypes(include=["int", "int64", "float", "float64"])

print("Object Columns:")
print(obj_cols.columns)
print("\nInteger Columns:")
print(int_cols.columns)


Do the same for all non-object dtype to note where they fall between "int", "int64", "float", "float64"

In [ ]:
obj_cols = int_cols.select_dtypes(include=['object'])

int_cols = int_cols.select_dtypes(include=["int", "int64", "float", "float64"])

print("Object Columns:")
print(obj_cols.columns)
print("\nInteger Columns:")
print(int_cols.columns)

In [ ]:
int_cols

#Random Forest Regression to predict AI Score

In [ ]:
# Assuming int_cols is your DataFrame with integer format ai_score
A = int_cols.drop(columns=["ai_score"])
B = int_cols["ai_score"]

# Splitting the data into training and testing sets
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.3, random_state=42)

# Building the Random Forest regression model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(A_train, B_train)

# Feature importances
feature_importances = rf_regressor.feature_importances_

# Getting the top 50 features
top_50_indices = feature_importances.argsort()[-50:][::-1]
top_50_features = A.columns[top_50_indices]
top_50_importances = feature_importances[top_50_indices]

# Plotting the top 50 features
plt.figure(figsize=(20, 12))
plt.barh(top_50_features, top_50_importances)
plt.xlabel('Feature Importance')
plt.title('Top 50 Features Using Random Forest Regression')
plt.gca().invert_yaxis()  # Invert y-axis to display most important features at the top
plt.savefig('/content/top_50_RFReg.pdf')
plt.show()

# Predicting on the test set
B_pred = rf_regressor.predict(A_test)

# Creating a DataFrame to display actual and predicted values side by side
results_df = pd.DataFrame({'Actual': B_test, 'Predicted': B_pred})
print(results_df)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculating Mean Absolute Error (MAE)
mae = mean_absolute_error(B_test, B_pred)

# Calculating Mean Squared Error (MSE)
mse = mean_squared_error(B_test, B_pred)

# Calculating R-squared (R2) score
r2 = r2_score(B_test, B_pred)

# Printing the evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2) Score:", r2)


In [ ]:
# Saving evaluation metrics to a file
with open('/content/evaluation_metrics.txt', 'w') as f:
    f.write("Mean Absolute Error (MAE): {}\n".format(mae))
    f.write("Mean Squared Error (MSE): {}\n".format(mse))
    f.write("R-squared (R2) Score: {}\n".format(r2))

# Visualizing actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(B_test, B_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted Values for Random Forest Regression')
plt.savefig('/content/actual_vs_predicted_RFReg.png')
plt.show()


#KNN Classifier to predict AI Ranking

In [ ]:
# Convert ai_ranking to integer format
int_cols['ai_ranking'] = int_cols['ai_ranking'].astype(int)

# Split the data into features (A) and target variable (B)
A = int_cols.drop(columns=["ai_ranking"])
B = int_cols["ai_ranking"]

# Split the data into training and testing sets
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.3, random_state=42)

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors

# Train the KNN model
knn_classifier.fit(A_train, B_train)

# Predictions
y_pred_train = knn_classifier.predict(A_train)
y_pred_test = knn_classifier.predict(A_test)

# Evaluate the model
# Accuracy
train_accuracy = accuracy_score(B_train, y_pred_train)
test_accuracy = accuracy_score(B_test, y_pred_test)

# Classification report
train_classification_report = classification_report(B_train, y_pred_train)
test_classification_report = classification_report(B_test, y_pred_test)

print(f'Train Accuracy: {train_accuracy}')
print(f'Test Accuracy: {test_accuracy}')

print("Train Classification Report:")
print(train_classification_report)

print("Test Classification Report:")
print(test_classification_report)


In [ ]:
metrics_filename = '/content/evaluation_metrics.txt'
classification_report_train_filename = '/content/classification_report_train.txt'
classification_report_test_filename = '/content/classification_report_test.txt'

with open(metrics_filename, 'w') as f:
    f.write(f'Train Accuracy: {train_accuracy}\n')
    f.write(f'Test Accuracy: {test_accuracy}\n')

with open(classification_report_train_filename, 'w') as f:
    f.write("Train Classification Report:\n")
    f.write(train_classification_report)

with open(classification_report_test_filename, 'w') as f:
    f.write("Test Classification Report:\n")
    f.write(test_classification_report)

# Zip the files
import zipfile
with zipfile.ZipFile('/content/evaluation_metrics_and_reports.zip', 'w') as zipf:
    for file in [metrics_filename, classification_report_train_filename, classification_report_test_filename]:
        zipf.write(file, os.path.basename(file))

In [ ]:
# Create a DataFrame with actual and predicted rankings
compard_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})

# Display the DataFrame
print(compard_df)

compard_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})

# Reset index to have numerical index
compard_df.reset_index(drop=True, inplace=True)

# Plot the actual and predicted rankings
plt.figure(figsize=(10, 6))
compard_df.plot(kind='bar', figsize=(14, 8))
plt.title('Actual vs Predicted Rankings using KNN Classifier')
plt.xlabel('Sample Index')
plt.ylabel('Ranking')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('/content/actual_predicteed_KNNC.pdf')
plt.show()

Rank the top 50 important features for predicting AI Ranking

In [ ]:
# Define number of classes
num_classes = 5  # Adjust as needed

# Discretize the target variable into classes
B_train_discretized = np.digitize(B_train, np.arange(0, 101, 100/num_classes))
B_test_discretized = np.digitize(B_test, np.arange(0, 101, 100/num_classes))

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Train the model
rf_classifier.fit(A_train, B_train_discretized)

# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Get indices of top 50 features
top_50_indices = feature_importances.argsort()[-50:][::-1]

# Get top 50 feature names
top_50_features = A_train.columns[top_50_indices]

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x=feature_importances[top_50_indices], y=top_50_features)
plt.title('Top 50 Features Importance for AI Ranking Using KNN')
plt.xlabel('Feature Importance Score')
plt.ylabel('Feature')
plt.savefig('/content/feat_importance_KNNC.pdf')
plt.show()


#ANN Model Prediction for AI Score

standardize dataset for ANN

In [ ]:
std_cols = (int_cols - int_cols.mean()) / int_cols.std()

scaler = StandardScaler()
scaled_cols = scaler.fit_transform(std_cols)
sstar_df = pd.DataFrame(scaled_cols, columns=std_cols.columns)

In [ ]:
X =std_cols.drop(columns=["ai_score"])
y = std_cols["ai_score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

In [ ]:
model = Sequential([
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(1)  # No activation function for regression
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=36, validation_split=0.2, callbacks=[early_stopping])

# Evaluate model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('/content/loss_plot.pdf')
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
vy_test = y_test.values.reshape(-1, 1)
inv_ytest = scaler.inverse_transform(np.hstack((np.zeros((len(vy_test), std_cols.shape[1]-1)), vy_test)))

vy_pred = y_pred.reshape(-1, 1)
inv_ypred = scaler.inverse_transform(np.hstack((np.zeros((len(vy_pred), std_cols.shape[1]-1)), vy_pred)))

In [ ]:
y_testS = inv_ytest * int_cols['ai_score'].std() + int_cols['ai_score'].mean()
y_predS = inv_ypred * int_cols['ai_score'].std() + int_cols['ai_score'].mean()

comparison_df = pd.DataFrame({'Actual': y_testS.flatten(), 'Predicted': y_predS.flatten()})

# Step 3: Print/visualize the comparison
print(comparison_df)

import numpy as np

# Calculate difference between actual and predicted values
difference = np.abs(comparison_df['Actual'] - comparison_df['Predicted'])

# Define a color map
cmap = plt.get_cmap('coolwarm')  # Choose a colormap
norm = plt.Normalize(difference.min(), difference.max())  # Normalize the difference values
colors = cmap(norm(difference))

# compare using scatter plot
plt.scatter(comparison_df['Actual'], comparison_df['Predicted'], c=colors)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Scatterplot of Actual vs Predicted AI Score using ANN')
plt.colorbar(label='Absolute Difference')
plt.savefig('/content/scatter_plot_ANN.pdf')
plt.show()


# compare using line plot
plt.figure(figsize=(10, 5))
plt.plot(comparison_df['Actual'], label='Actual', color='blue')
plt.plot(comparison_df['Predicted'], label='Predicted', color='red')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Lineplot of Actual vs Predicted AI Score using ANN')
plt.legend()
plt.savefig('/content/line_plot_ANN.pdf')
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate MAE
mae = mean_absolute_error(y_testS, y_predS)

# Calculate MSE
mse = mean_squared_error(y_testS, y_predS)

# Calculate RMSE
rmse = mean_squared_error(y_testS, y_predS, squared=False)

# Calculate R-squared
r_squared = r2_score(y_testS, y_predS)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r_squared)

In [ ]:
# Get the weights of the connections between the input layer and the first hidden layer
weights = model.layers[0].get_weights()[0]

# Calculate feature importance by summing the absolute weights for each feature
feature_importance = np.sum(np.abs(weights), axis=1)

# Get the indices of the top 50 features
top_50_indices = feature_importance.argsort()[-50:][::-1]

# Get the names of the top 50 features
top_50_features = X_train.columns[top_50_indices]

# Plot the feature importance
plt.figure(figsize=(15, 10))
plt.barh(range(len(top_50_features)), feature_importance[top_50_indices], align='center')
plt.yticks(range(len(top_50_features)), top_50_features)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 50 Features Importance for AI Score Using ANN')
plt.gca().invert_yaxis()
plt.savefig('/content/top_50_ANN.pdf')
plt.show()


#Random Forest Regression Analysis for AI Score

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Feature importances
feature_importances = rf_model.feature_importances_

# Getting the top 50 features
top_50_indices = feature_importances.argsort()[-50:][::-1]
top_50_features = X.columns[top_50_indices]
top_50_importances = feature_importances[top_50_indices]

# Plotting the top 50 features
plt.figure(figsize=(20, 16))
plt.barh(top_50_features, top_50_importances)
plt.xlabel('Feature Importance')
plt.title('Top 50 Features')
plt.gca().invert_yaxis()  # Invert y-axis to display most important features at the top
plt.savefig('/content/RF_top_50_ANN.pdf')
plt.show()

# Predicting on the test set
y_pred = rf_model.predict(X_test)

# Creating a DataFrame to display actual and predicted values side by side
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results_df)

#KNN Regression Analysis for AI Ranking

In [ ]:
A =std_cols.drop(columns=["ai_ranking"])
B = std_cols["ai_ranking"]

A_train, A_test, B_train, B_test = train_test_split(A, B, test_size =0.3, random_state=42)

In [ ]:
print(B.shape)

In [ ]:
# Initialize the KNN regressor
knn_regressor = KNeighborsRegressor(n_neighbors=5)  # You can adjust the number of neighbors

# Train the KNN model
knn_regressor.fit(A_train, B_train)

# Predictions
y_pred_train = knn_regressor.predict(A_train)
y_pred_test = knn_regressor.predict(A_test)

# Evaluate the model
#RMSE
train_rmse = mean_squared_error(B_train, y_pred_train, squared=False)
test_rmse = mean_squared_error(B_test, y_pred_test, squared=False)

#MAE
train_mae = mean_absolute_error(B_train, y_pred_train)
test_mae = mean_absolute_error(B_test, y_pred_test)

# R-squared
train_r2 = r2_score(B_train, y_pred_train)
test_r2 = r2_score(B_test, y_pred_test)

print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')
print(f'Train MAE: {train_mae}')
print(f'Test MAE: {test_mae}')
print(f'Train R-squared: {train_r2}')
print(f'Test R-squared: {test_r2}')


In [ ]:
compare_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})
print(compare_df)

In [ ]:
import matplotlib.pyplot as plt

# Create a DataFrame with actual and predicted rankings
comparison_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})

# Reset index to have numerical index
comparison_df.reset_index(drop=True, inplace=True)

# Plot the actual and predicted rankings
plt.figure(figsize=(14, 8))

# Bar chart
plt.subplot(2, 1, 1)  # 2 rows, 1 column, plot 1
comparison_df.plot(kind='bar', ax=plt.gca())
plt.title('Actual vs Predicted Rankings')
plt.xlabel('Sample Index')
plt.ylabel('Ranking')
plt.xticks(rotation=45)

# Scatter plot with line
plt.subplot(2, 1, 2)  # 2 rows, 1 column, plot 2
plt.scatter(B_train, y_pred_train, label='Training data', color='blue', alpha=0.5)
plt.scatter(B_test, y_pred_test, label='Test data', color='red', alpha=0.5)
plt.plot([B_train.min(), B_train.max()], [B_train.min(), B_train.max()], 'k--', lw=2)  # Diagonal line
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted values for AI Ranking Using KNN Regression')
plt.savefig('/content/top_50_KNN_Regression.pdf')
plt.legend()

plt.tight_layout()

plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(B_train, y_pred_train, label='Training data', color='blue', alpha=0.5)
plt.scatter(B_test, y_pred_test, label='Test data', color='red', alpha=0.5)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted values for KNN Model')
plt.legend()
plt.show()

In [ ]:
# Calculate the absolute difference in predictions when a feature value is perturbed
train_feature_importances = np.mean(np.abs(knn_regressor.predict(A_train) - knn_regressor.predict(A_train.values + 0.01)), axis=0)
test_feature_importances = np.mean(np.abs(knn_regressor.predict(A_test) - knn_regressor.predict(A_test.values + 0.01)), axis=0)

# Get the indices of features sorted by importance
train_sorted_indices = train_feature_importances.argsort()[::-1]
test_sorted_indices = test_feature_importances.argsort()[::-1]

# Get the top 50 features and their importance scores
top_train_features = A_train.columns[train_sorted_indices][:50]
top_train_scores = train_feature_importances[train_sorted_indices][:50]

top_test_features = A_test.columns[test_sorted_indices][:50]
top_test_scores = test_feature_importances[test_sorted_indices][:50]

# Plot the top 50 features and their importance scores
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.barh(top_train_features, top_train_scores, color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.title('Top 50 Features Importance - Train Data')
plt.gca().invert_yaxis()  # Invert y-axis to display in descending order

plt.subplot(2, 1, 2)
plt.barh(top_test_features, top_test_scores, color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.title('Top 50 Features Importance - Test Data')
plt.gca().invert_yaxis()  # Invert y-axis to display in descending order

plt.tight_layout()
plt.show()
